In [3]:
from web3 import Web3
import json
import os
import secrets

In [9]:
# Configuration
AVALANCHE_FUJI_RPC = "https://api.avax-test.network/ext/bc/C/rpc"
CONTRACT_ADDRESS = "0x85ac2e065d4526FBeE6a2253389669a12318A412"
PRIVATE_KEY = "0x82429e0e75ae4201759386e760a55601f6280a8c025366f90f07460915dc2ff4"
WALLET_ADDRESS = "0x86a11d271dA11aa145cAE9f8396b09Aa4C0530Bb"

# Connect to Web3
w3 = Web3(Web3.HTTPProvider(AVALANCHE_FUJI_RPC))

if not w3.is_connected():
    raise Exception("Failed to connect to Avalanche Fuji Testnet")

# Load contract ABI from uploaded file
abi_path = "NFT.abi"
with open(abi_path, "r") as abi_file:
    contract_abi = json.load(abi_file)

# Instantiate contract
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=contract_abi)

def claim_nft():
    account = w3.eth.account.from_key(PRIVATE_KEY)
    nonce = w3.eth.get_transaction_count(WALLET_ADDRESS)
    random_nonce = secrets.token_bytes(32)  # Generate a random 32-byte nonce
    
    # Pass both arguments to the claim function
    txn = contract.functions.claim(WALLET_ADDRESS, random_nonce).build_transaction({
        'from': WALLET_ADDRESS,
        'gas': 250000,
        'gasPrice': w3.to_wei('25', 'gwei'),
        'nonce': nonce
    })
    
    signed_txn = w3.eth.account.sign_transaction(txn, private_key=PRIVATE_KEY)
    
    # Use the correct attribute name for the raw transaction
    txn_hash = w3.eth.send_raw_transaction(signed_txn.raw_transaction)
    
    return w3.to_hex(txn_hash)

# Execute minting
txn_hash = claim_nft()
print(f"NFT Claimed! Transaction Hash: {txn_hash}")

NFT Claimed! Transaction Hash: 0xb57c8fb9e8c61d29344bdbb0de75d45742e3988a1ef65aa1f140e8d35a6f9f7a
